# Optimizing Fantasy Football Auction Drafts utilizing Linear Programming
Fantasy football auction drafts can be formulated as a Linear Programming optimization problem. The simplest form assumes that the projected points and bid value stays constant throughout the draft. While that assumption is likely to hold for projected points, it is not for the bid value. Bid value is a dynamic variable that is dependent on team budgets, team needs, and personal biases. However, for pre-draft analysis, this is a valid assumption as the output is useful.

## LP Problem Formulation

Let binary decision variables  be $X_{i}$ where $X_{i} \in {[0, 1]}$ 
indicating whether player $X_{i}$ is drafted.

$$\begin{aligned}
 \text{Maximize} && \sum{p_{i}x_{i}}\\
 \text{subject to:}\\
 \text{QBs:} && x_{i\in QB} = 1 && (1)\\
 \text{RBs:} && x_{i\in RB} >= 2 && (2) \\
 \text{WRs:}&& x_{i\in WR} >= 2 && (3)  \\
 \text{TEs:}&& x_{i\in TE} >= 2 && (4) \\
 \text{K:}&& x_{i\in K} = 1 && (5) \\
 \text{Flex:}&& x_{i\in RB} + x_{i\in WR} + x_{i\in TE} <= 6 && (6) \\
 \text{Def:}&& x_{i\in DST} = 1 && (7) \\
 \text{Budget:}&& x_{i}*c_{i} <= 200 && (8) \\
\end{aligned}$$

Where:
- Constraints (1)-(7) are positional constraints (i.e., cannot start more than 1 QB)
- $p_{i}$ is the projected value of player $x_{i}$ over the course of a season
- $c_{i}$ is the projected value bid of player $x_{i}$

In [555]:
# Set up
import pandas as pd
import gurobipy as grb

def clean_currency(x):
    """ If the value is a string, then remove currency symbol and delimiters
    otherwise, the value is numeric and can be converted
    """
    if isinstance(x, str):
        return(x.replace('$', '').replace(',', ''))
    return(x)

def merge_with_bid(positional_data, bid_data, pos):
    value_data = positional_data.merge(bid_data, how = 'left', left_on = 'Player', right_on = 'name')
    value_data['Value'] = value_data['Value'].apply(clean_currency).astype('float')
    value_data = value_data[value_data['Value'] >= 0]
    value_data = pd.DataFrame({'player': value_data['Player'], 'pts': value_data['FPTS'], 'bid': value_data['Value'], 'pos': pos})
    return value_data

In [557]:
# Read in projected bid data and remove team names from player name
projected_bid = pd.read_csv('/Users/dsung/fantasy_football/data/projected_bid.csv')
projected_bid['name'] = projected_bid['Overall'].str.replace(r' \(.*', '', regex=True)

# Read in positional data
data = {}
data['qb'] = pd.read_csv('/Users/dsung/fantasy_football/data/FantasyPros_Fantasy_Football_Projections_QB.csv')
data['rb'] = pd.read_csv('/Users/dsung/fantasy_football/data/FantasyPros_Fantasy_Football_Projections_RB.csv')
data['wr'] = pd.read_csv('/Users/dsung/fantasy_football/data/FantasyPros_Fantasy_Football_Projections_WR.csv')
data['te'] = pd.read_csv('/Users/dsung/fantasy_football/data/FantasyPros_Fantasy_Football_Projections_TE.csv')
data['k'] = pd.read_csv('/Users/dsung/fantasy_football/data/FantasyPros_Fantasy_Football_Projections_K.csv')
data['dst'] = pd.read_csv('/Users/dsung/fantasy_football/data/FantasyPros_Fantasy_Football_Projections_DST.csv')


positions = ['qb', 'rb', 'wr', 'te', 'k', 'dst']
pos_data = {}
for p in positions:
    pos_data[p] = merge_with_bid(data[p], projected_bid, p)

for p in positions:
    if p == positions[0]:
        player_data = pos_data[p]
    else:
        player_data = pd.concat([player_data, pos_data[p]])

pos_cap_dict = {'qb': 1, 'rb':2, 'wr': 2, 'te': 1, 'k': 1, 'dst': 1, 'flex': 1}


# position player list
positions = ['qb', 'rb', 'wr', 'te', 'k', 'dst']
flex_positions = ['rb', 'wr', 'te']

pos_dict = {}
for p in positions:
    pos_dict[p] = player_data[player_data['pos'] == p]['player']

In [558]:
mod.dispose()

# constants
initial_budget = 200
total_spent = 0
budget = initial_budget - total_spent

# Create decision variables
mod=grb.Model()
x={}
for p in positions:
    tmp_df = player_data[player_data['pos']==p]
    for index, row in tmp_df.iterrows():
        i = row['player']
        x[i, p]=mod.addVar(lb=0,name='x[{0},{1}]'.format(i, p), vtype=grb.GRB.BINARY)

# position capacity constraint
for p in positions:
    if p in flex_positions:
        mod.addConstr(sum(x[i, p] for i in player_data[player_data['pos']==p]['player']) >= pos_cap_dict[p],name=p + ' positional constraint')    
    else:
        mod.addConstr(sum(x[i, p] for i in player_data[player_data['pos']==p]['player']) == pos_cap_dict[p],name=p + ' positional constraint')

# flex position constraint
flex_constraint = {}
flex_max = sum(pos_cap_dict[p] for p in flex_positions) + pos_cap_dict['flex']
for f in flex_positions:
    flex_constraint[f] = sum(x[i, f] for i in pos_dict[f])

mod.addConstr(sum(flex_constraint[f] for f in flex_positions) <= flex_max, name = 'flex constraint')



# budget constraint
budgets = {}
for p in positions:
    budgets[p] = sum(x[i, p] * player_data[(player_data['player'] == i) & (player_data['pos'] == p)]['bid'] for i in pos_dict[p])
mod.addConstr(sum(budgets[p] for p in positions) <= budget, name = 'budget_constraint')


# optimize function
optimize = {}
for p in positions:
    optimize[p] = sum(x[i, p] * player_data[(player_data['player'] == i) & (player_data['pos'] == p)]['pts'] for i in pos_dict[p])

mod.setObjective(sum(optimize[p] for p in positions),sense=grb.GRB.MAXIMIZE) 
mod.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 8 rows, 324 columns and 727 nonzeros
Model fingerprint: 0x6aab50fd
Variable types: 0 continuous, 324 integer (324 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [3e+01, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective 1300.8000000
Presolve removed 1 rows and 81 columns
Presolve time: 0.00s
Presolved: 7 rows, 243 columns, 615 nonzeros
Variable types: 0 continuous, 243 integer (239 binary)
Found heuristic solution: objective 1823.7000000

Root relaxation: objective 1.857374e+03, 10 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1857.37407    0    2 1823

In [562]:
optimal_draft = pd.DataFrame({'players': player_data['player'], 'draft_status': mod.X, 'pos': player_data['pos'], 'pts': player_data['pts'], 'bid': player_data['bid']})
optimal_draft = optimal_draft[optimal_draft['draft_status'] == 1].sort_values('bid', ascending = False)
optimal_draft.to_csv('/Users/dsung/fantasy_football/output/optimal_team.csv')
print(optimal_draft)

                players  draft_status  pos    pts   bid
1   Christian McCaffrey           1.0   rb  269.5  49.0
2         Austin Ekeler           1.0   rb  261.0  46.0
1    Patrick Mahomes II           1.0   qb  383.9  33.0
9     Amon-Ra St. Brown           1.0   wr  208.3  28.0
3        T.J. Hockenson           1.0   te  157.8  20.0
25         Chris Godwin           1.0   wr  170.5  12.0
34      Diontae Johnson           1.0   wr  154.3   9.0
0        Dallas Cowboys           1.0  dst  119.3   2.0
0         Zane Gonzalez           1.0    k  130.5   1.0
